In [0]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
%cd '/gdrive/My Drive/'

/gdrive/My Drive


In [0]:
!ls -l analysis/recursion/data/

total 47869146
drwx------ 3 root root        4096 Sep  9 13:52 data_pkl
drwx------ 4 root root        4096 Sep  9 00:35 data_raw
-rw------- 1 root root    18449704 Sep  8 10:08 pixel_stats.csv.zip
-rw------- 1 root root       35620 Sep  8 10:08 recursion_dataset_license.pdf
-rw------- 1 root root      367018 Sep  8 10:08 sample_submission.csv
-rw------- 1 root root      114364 Sep  8 10:08 test_controls.csv
-rw------- 1 root root      574862 Sep  8 10:08 test.csv
-rw------- 1 root root 16872118408 Sep  8 10:23 test.zip
drwx------ 2 root root        4096 Sep  9 03:06 train2
-rw------- 1 root root      208866 Sep  8 10:08 train_controls.csv
-rw------- 1 root root     1203816 Jun 26 08:01 train.csv
-rw------- 1 root root      240888 Sep  8 10:08 train.csv.zip
-rw------- 1 root root 32124677223 Sep  8 10:26 train.zip


In [0]:
# !git clone https://github.com/recursionpharma/rxrx1-utils

In [0]:
# %cd analysis/recursion/data

In [0]:
# !mkdir /root/.kaggle/
# !cp /gdrive/My\ Drive/analysis/kaggle/kaggle.json /root/.kaggle/

In [0]:
# !kaggle competitions download -c recursion-cellular-image-classification -f train.zip -p train2.zip

In [0]:
# !mkdir data_raw/train2

In [27]:
!unzip -qo -d data_raw/train2 train2/train.zip 

error:  zipfile read error


## config

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import os
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import math
import cv2

from tqdm import tqdm
import joblib

from sklearn.model_selection import train_test_split

In [0]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

from torch.utils.data import Dataset, DataLoader

In [0]:
# !pip install pretrainedmodels
# import pretrainedmodels

In [0]:
sys.path.append('analysis/recursion/src')

from common.logger import create_logger, get_logger
from model.loss import ArcMarginProduct
from common.util import str_stats
from model.metrics import AverageMeter
from model.model_util import save_checkpoint, load_checkpoint

In [0]:
# sys.path.append('analysis/recursion/src/rxrx1-utils')
# import rxrx.io as rio

ModuleNotFoundError: ignored

In [0]:
IMG_SIZE = (512, 512)
INPUT_SIZE = (256, 256)
TEST_INPUT_SIZE = (256, 256) #(1024, 1024)

INPUT = 'analysis/recursion/data/'
TRAIN_PATH = os.path.join(INPUT, 'train.csv')
TEST_PATH = os.path.join(INPUT, 'test.csv')
TRAIN_IMG_PATH = os.path.join(INPUT, 'data_raw', 'train2')
TEST_IMG_PATH = os.path.join(INPUT, 'data_raw', 'test')

N_SAMPLES = 5
RUN_TTA = False

In [0]:
N_CLASSES = 1108

BATCH_SIZE_TRAIN = 32
BATCH_SIZE_TEST = 128
NUM_WORKERS = 8
PRINT_FREQ = 10
ITER_PER_CYCLE = 20
EPOCHS = 5 + ITER_PER_CYCLE * 4

ADAM_LR = 5e-4
SGD_LR = 1e-3
MIN_LR = 1e-3
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4

USE_PRETRAINED = False
PRETRAIN_PATH = 'analysis/recursion/models/res34_unet_1/best_model.pth'

DROPOUT_RATE = 0.2
LATENT_DIM = 512
TEMPERATURE = 60
MARGIN = 0.5

In [0]:
create_logger('recursion.log')
get_logger().info('start kernel')

[INFO]2019-09-09 14:03:48,136:main:start kernel


# eda

In [0]:
# df_train = pd.read_csv(TRAIN_PATH)
# df_train.head()

In [0]:
# t = rio.load_site('train', 'RPE-05', 3, 'D19', 2)
# t.shape

In [0]:
'''
def load_site(dataset,
              experiment,
              plate,
              well,
              site,
              channels=DEFAULT_CHANNELS,
              base_path=DEFAULT_IMAGES_BASE_PATH):
'''

# helper_func

In [0]:
def train_valid_split_v1(df, valid_ratio):
    df_trn, df_val = train_test_split(df, 
                                       stratify=df['sirna'], 
                                       test_size=valid_ratio, 
                                       random_state=2019, 
                                       shuffle=True)
    return df_trn, df_val

# metrics

# dataset

In [0]:
import cv2
import albumentations as A
import albumentations.pytorch as ATorch

alb_trn_trnsfms = A.Compose([
    # A.CLAHE(p=1),
    A.Rotate(limit=10, p=1),
    # A.RandomSizedCrop((IMG_SIZE[0]-32, IMG_SIZE[0]-10), *INPUT_SIZE),    
    A.RandomCrop(*INPUT_SIZE),
    # A.HueSaturationValue(val_shift_limit=20, p=0.5),
    # A.RandomBrightnessContrast(),        
    # A.Resize(*INPUT_SIZE),    
    A.Normalize(
        mean=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
    ),
    ATorch.transforms.ToTensor()
], p=1)
    
'''
    A.Normalize(
        mean=[0.485, 0.456, 0.406, 0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225, 0.229, 0.224, 0.225],
        always_apply=False
    ),
'''


alb_val_trnsfms = A.Compose([
    # A.CLAHE(p=1),
    # A.Resize(*INPUT_SIZE),
    A.CenterCrop(*INPUT_SIZE),
    A.Normalize(
        mean=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5],  
    ),
    ATorch.transforms.ToTensor()
], p=1)

alb_tst_trnsfms = A.Compose([
    # A.CLAHE(),
    A.Resize(*TEST_INPUT_SIZE),
    A.Normalize(
        mean=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5],          
    ),
    ATorch.transforms.ToTensor()
], p=1)

In [0]:
class CellerDataset(Dataset):    
    target_name = 'sirna'
    image_name = ''
    id_name = 'id_code'
    site = 1

    def __init__(self, df, image_dir, transform, mode):
        self.df_org = df.copy()        
        self.image_dir = image_dir        
        self.transform = transform
        self.mode = mode

        # Random Selection
        if mode == 'train':
            self.update()
        elif mode in ['valid', 'predict']:
            self.df_selected = self.df_org
        else:
            raise ValueError('Unexpected mode: %s' % mode)
    
    def __len__(self):
        return self.df_selected.shape[0]
    
    def __getitem__(self, idx):        
        # image_names = get_image_name(self.df_selected, idx, self.site)
        # image = self.__load_image(image_names)
        image = self.__load_image_with_rxrx(self.df_selected, idx)
        if self.mode in ['train', 'valid']:
            label =  self.df_selected.iloc[idx][self.target_name]            
        elif self.mode == 'predict':
            label = -1
        return image, torch.Tensor([label])
    
    def __load_image_with_rxrx(self, df, idx):
        dataset = 'train' if self.mode in ['train', 'valid'] else 'test'

        rcd = df.iloc[idx]
        experiment = rcd['experiment']
        plate = rcd['plate']
        well = rcd['well']
        image = rio.load_site('train', experiment, plate, well, 1)
        
        augmented = self.transform(image=image)
        image = augmented['image']
        return image

    def __load_image(self, image_names):
        """
        Parameters
        ----------
        image_names: list of str
        """

        images = []
        for image_name in image_names:
            image_path = os.path.join(self.image_dir, image_name)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            
            if image is None:
                raise ValueError('Not found image: %s' % image_path)
            images.append(image)
        images = np.stack(images, axis=2)
        
        augmented = self.transform(image=images)
        image = augmented['image']
        return image

    def update(self):
        if self.mode != 'train':
            raise ValueError('CellerDataset is not train mode.')
        self.df_selected = self.random_selection(N_SAMPLES)

    def random_selection(self, n_samples):                        
        g = self.df_org.groupby(self.target_name)[self.id_name]
        # selected = []
        # selected.append(np.random.choice(g.get_group(0).tolist(), n_samples, replace=False))
        # selected.append(np.random.choice(g.get_group(1).tolist(), n_samples, replace=False))
        selected = [np.random.choice(g.get_group(i).tolist(), n_samples, replace=False) for i in range(N_CLASSES)]
        selected = np.concatenate(selected, axis=0)

        df_new = pd.DataFrame({self.id_name: selected})
        df_new = df_new.merge(self.df_org, on=self.id_name, how='left')
        get_logger().info('num of selected_images: %d' % len(df_new))

        return df_new


def get_image_name(df, idx, site):
    """
    Returns
    -------
    file_paths: list
        image path list of 6 channels
    """
    rcd = df.iloc[idx]
    experiment = rcd['experiment']
    plate = 'Plate%d' % rcd['plate']
    well = rcd['well']

    file_names = ['%s_s%d_w%d.png' % (well, site, i+1) for i in range(6)]
    file_paths = [os.path.join(experiment, plate, file_name) for file_name in file_names]
    return file_paths


# model/loss

In [0]:
class ResNet(nn.Module):
    def __init__(self, dropout_rate, latent_dim, temperature, m):
        super(ResNet, self).__init__()
        # self.resnet = torchvision.models.resnet50(pretrained=True)
        self.resnet = torchvision.models.resnet34(pretrained=True)
        # self.resnet = torchvision.models.resnet18(pretrained=True)
        n_out_channels = 512  # resnet18, 34: 512, resnet50: 512*4

        conv1 = self.resnet.conv1
        self.resnet.conv1 = nn.Conv2d(in_channels=6,
                                      out_channels=conv1.out_channels,
                                      kernel_size=conv1.kernel_size,
                                      stride=conv1.stride,
                                      padding=conv1.padding,
                                      bias=conv1.bias)

        # copy pretrained weights
        self.resnet.conv1.weight.data[:,:3,:,:] = conv1.weight.data
        self.resnet.conv1.weight.data[:,3:,:,:] = conv1.weight.data

        '''
        trained_kernel = self.resnet.conv1.weight
        new_conv = nn.Conv2d(6, 64, kernel_size=7, stride=2, padding=3, bias=False)
        with torch.no_grad():
            new_conv.weight[:,:] = torch.stack([torch.mean(trained_kernel, 1)]*6, dim=1)
        self.resnet.conv1 = new_conv
        '''

        # FC        
        self.norm1 = nn.BatchNorm1d(n_out_channels)
        self.drop1 = nn.Dropout(dropout_rate)
        # FC
        self.fc = nn.Linear(n_out_channels, latent_dim)
        self.norm2 = nn.BatchNorm1d(latent_dim)
        self.arc = ArcMarginProduct(latent_dim, N_CLASSES, s=temperature, m=m, easy_margin=False)

    def forward(self, x, label=None):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)

        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)
        
        # GAP
        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = x.view(x.size(0), -1)
        x = self.norm1(x)
        x = F.relu(x)
        x = self.drop1(x)
        # FC
        x = self.fc(x)
        # x = self.norm2(x)
        # Arc
        x = self.arc(x, label)

        return x

In [0]:
class SEResNet(nn.Module):
    def __init__(self, dropout_rate, latent_dim, temperature, m):
        super(SEResNet, self).__init__()

        senet = pretrainedmodels.__dict__['se_resnext50_32x4d'](
            num_classes=1000, pretrained='imagenet')
        self.layer0 = senet.layer0
        self.layer1 = senet.layer1
        self.layer2 = senet.layer2
        self.layer3 = senet.layer3
        self.layer4 = senet.layer4

        self.norm1 = nn.BatchNorm1d(512*4)
        self.drop1 = nn.Dropout(dropout_rate)
        # FC
        self.fc = nn.Linear(512*4, latent_dim)        
        # self.norm2 = nn.BatchNorm1d(output_neurons)
        self.arc = ArcMarginProduct(latent_dim, 2, s=temperature, m=m, easy_margin=False)

    def forward(self, x, label=None):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        # GAP
        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = x.view(x.size(0), -1)
        x = self.norm1(x)
        x = F.relu(x)
        x = self.drop1(x)
        # FC
        x = self.fc(x)
        # x = self.norm2(x)
        # Arc
        x = self.arc(x, label)

        return x


## warmup

In [0]:
from torch.optim.lr_scheduler import _LRScheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau


class GradualWarmupScheduler(_LRScheduler):
    """ Gradually warm-up(increasing) learning rate in optimizer.
    Proposed in 'Accurate, Large Minibatch SGD: Training ImageNet in 1 Hour'.

    Args:
        optimizer (Optimizer): Wrapped optimizer.
        multiplier: target learning rate = base lr * multiplier
        total_epoch: target learning rate is reached at total_epoch, gradually
        after_scheduler: after target_epoch, use this scheduler(eg. ReduceLROnPlateau)
    """

    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        self.multiplier = multiplier
        if self.multiplier <= 1.:
            raise ValueError('multiplier should be greater than 1.')
        self.total_epoch = total_epoch
        self.after_scheduler = after_scheduler
        self.finished = False
        super().__init__(optimizer)

    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]

        return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]

    def step_ReduceLROnPlateau(self, metrics, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
        self.last_epoch = epoch if epoch != 0 else 1  # ReduceLROnPlateau is called at the end of epoch, whereas others are called at beginning
        if self.last_epoch <= self.total_epoch:
            warmup_lr = [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]
            for param_group, lr in zip(self.optimizer.param_groups, warmup_lr):
                param_group['lr'] = lr
        else:
            if epoch is None:
                self.after_scheduler.step(metrics, None)
            else:
                self.after_scheduler.step(metrics, epoch - self.total_epoch)

    def step(self, epoch=None, metrics=None):
        if type(self.after_scheduler) != ReduceLROnPlateau:
            if self.finished and self.after_scheduler:
                if epoch is None:
                    self.after_scheduler.step(None)
                else:
                    self.after_scheduler.step(epoch - self.total_epoch)
            else:
                return super(GradualWarmupScheduler, self).step(epoch)
        else:
            self.step_ReduceLROnPlateau(metrics, epoch)

# train

In [0]:
from model.metrics import accuracy

def train_one_epoch(epoch,
                        model,
                        loader,
                        criterion,
                        optimizer):
    loss_meter = AverageMeter()
    top1_meter = AverageMeter()
    top5_meter = AverageMeter()

    get_logger().info('[Start] epoch: %d' % epoch)
    get_logger().info('lr: %f' %
                      optimizer.state_dict()['param_groups'][0]['lr'])
    loader.dataset.update()
    # train phase
    model.train()
    for i, data in enumerate(tqdm(loader)):
        img, label = data
        img, label = img.cuda(), label.cuda().long()
        with torch.set_grad_enabled(True):        
            logit = model(img, label)
            loss = criterion(logit, label.squeeze())

            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_meter.update(loss.item(), img.size(0))            
            score_top1, score_top5 = accuracy(logit.detach(), label.detach(), topk=(1, 5))
            top1_meter.update(score_top1.item(), img.size(0))
            top5_meter.update(score_top5.item(), img.size(0))

        # print
        if i % PRINT_FREQ == 0:
            logit_cpu = logit.view(img.size(0), -1).detach().cpu()
            get_logger().info('\n' + str_stats(logit_cpu[0].numpy()))  
            softmaxed = F.softmax(logit_cpu, dim=1)       
            get_logger().info('\n' + str_stats(softmaxed[0].numpy()))
            get_logger().info('train: %d loss: %f top1: %f top5: %f (just now)' %
                              (i, loss_meter.val, top1_meter.val, top5_meter.val))
            get_logger().info('train: %d loss: %f top1: %f top5: %f' %
                              (i, loss_meter.avg, top1_meter.avg, top5_meter.avg))
            
    get_logger().info("Epoch %d/%d train loss %f top1 %f top5 %f" 
                      % (epoch, EPOCHS, loss_meter.avg, top1_meter.avg, top5_meter.avg))

    return loss_meter.avg, top1_meter.avg, top5_meter.avg

def validate_one_epoch(epoch,
                        model,                   
                        loader,
                        criterion):
    loss_meter = AverageMeter()
    top1_meter = AverageMeter()
    top5_meter = AverageMeter()   

    # validate phase
    model.eval()
    for i, data in enumerate(tqdm(loader)):
        img, label = data
        img, label = img.cuda(), label.cuda().long()
        with torch.no_grad():
            logit = model(img)           

            loss = criterion(logit, label.squeeze())
            loss_meter.update(loss.item(), img.size(0))
            score_top1, score_top5 = accuracy(logit.detach(), label, topk=(1, 5))
            top1_meter.update(score_top1.item(), img.size(0))        
            top5_meter.update(score_top5.item(), img.size(0))        

    get_logger().info("Epoch %d/%d valid loss %f top1 %f top5 %f" 
                      % (epoch, EPOCHS, loss_meter.avg, top1_meter.avg, top5_meter.avg))

    return loss_meter.avg, top1_meter.avg, top5_meter.avg

In [0]:
from model.loss import FocalLoss

def train():
    # Load csv
    df_train = pd.read_csv(TRAIN_PATH)        
    df_trn, df_val = train_valid_split_v1(df_train, valid_ratio=0.05)
    get_logger().info('train size: %d valid size: %d' % (len(df_trn), len(df_val)))

    train_dataset = CellerDataset(df_trn, TRAIN_IMG_PATH, alb_trn_trnsfms, mode='train')
    train_loader = DataLoader(train_dataset,
                                batch_size=BATCH_SIZE_TRAIN,
                                num_workers=NUM_WORKERS,
                                pin_memory=True,
                                drop_last=True,
                                shuffle=True
                                )
    valid_dataset = CellerDataset(df_val, TRAIN_IMG_PATH, alb_val_trnsfms, mode='valid')
    valid_loader = DataLoader(valid_dataset,
                              batch_size=BATCH_SIZE_TRAIN,
                              num_workers=NUM_WORKERS,
                              pin_memory=True,
                              drop_last=False,
                              shuffle=False
                              )    
    model, criterion, optimizer, scheduler = init_model()
    start_epoch = 0
    if USE_PRETRAINED:
        start_epoch, model, optimizer, scheduler, _ = load_checkpoint(model, optimizer, scheduler, CLF_PRETRAIN_PATH)

    get_logger().info('[Start] Recursion Celler Training')
    best_score = 0
    train_history = {'loss': [], 'top1': [], 'top5': []}
    valid_history = {'loss': [], 'top1': [], 'top5': []}
    for epoch in range(start_epoch + 1, EPOCHS + 1):
        train_loss, train_top1, train_top5 = train_one_epoch(epoch, model, train_loader, criterion, optimizer)
        valid_loss, valid_top1, valid_top5 = validate_one_epoch(epoch, model, valid_loader, criterion)

        train_history['loss'].append(train_loss)
        train_history['top1'].append(train_top1)
        train_history['top5'].append(train_top1)
        valid_history['loss'].append(valid_loss)
        valid_history['top1'].append(valid_top1)
        valid_history['top5'].append(valid_top5)
        
        valid_score = valid_top1
        is_best = valid_score > best_score
        if is_best:
            best_score = valid_score
        get_logger().info('best score (%f) at epoch (%d)' % (valid_score, epoch))
        save_checkpoint({
            'epoch': epoch,
            'state_dict': model.state_dict(),            
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }, is_best)
        # move scheduler.step to here
        scheduler.step()

    return train_history, valid_history

In [0]:
def init_model():
    torch.backends.cudnn.benchmark = True
    get_logger().info('Initializing classification model...')
    model = ResNet(dropout_rate=DROPOUT_RATE, 
                   latent_dim=LATENT_DIM, 
                   temperature=TEMPERATURE,
                   m=MARGIN).cuda()
    # model = SEResNet(dropout_rate=0.3, latent_dim=512, temperature=1.0, m=MARGIN).cuda()
    criterion = FocalLoss(gamma=2).cuda()    
    '''
    optimizer = torch.optim.SGD([{'params': model.parameters()}],
                                lr=SGD_LR, 
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, EPOCHS, MIN_LR)
    '''
    optimizer = optim.Adam([{'params': model.parameters()}], lr=ADAM_LR)
    mile_stones = [10, 20, 30, 40]
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, mile_stones, gamma=0.5, last_epoch=-1)                    
    
    return model, criterion, optimizer, scheduler

# main

In [0]:
train()

[INFO]2019-09-09 12:12:40,176:main:train size: 34689 valid size: 1826
[INFO]2019-09-09 12:12:40,612:main:num of selected_images: 5540
[INFO]2019-09-09 12:12:40,615:main:Initializing classification model...
[INFO]2019-09-09 12:12:44,286:main:[Start] Recursion Celler Training
[INFO]2019-09-09 12:12:44,289:main:[Start] epoch: 1
[INFO]2019-09-09 12:12:44,292:main:lr: 0.000500
[INFO]2019-09-09 12:12:44,658:main:num of selected_images: 5540
  0%|          | 0/173 [00:00<?, ?it/s][INFO]2019-09-09 12:13:51,054:main:
    count      mean       std  ...       50%       75%       max
0  1108.0 -0.026315  2.744862  ...  0.094188  1.696812  9.360255

[1 rows x 8 columns]
[INFO]2019-09-09 12:13:51,159:main:
    count      mean       std  ...       50%       75%       max
0  1108.0  0.000903  0.010581  ...  0.000028  0.000137  0.291325

[1 rows x 8 columns]
[INFO]2019-09-09 12:13:51,267:main:train: 0 loss: 38.936111 top1: 0.000000 top5: 0.000000 (just now)
[INFO]2019-09-09 12:13:51,275:main:train: 0 l

KeyboardInterrupt: ignored

In [0]:
# df_train = pd.read_csv(TRAIN_PATH)
# df_trn, df_val = train_valid_split_v1(df_train, valid_ratio=0.1)

In [0]:
# dataset = CellerDataset(df_trn, TRAIN_IMG_PATH, alb_trn_trnsfms, mode='train')

In [0]:
'''
fig, axes = plt.subplots(2, 3, figsize=(24, 16))

for i, ax in enumerate(axes.flatten()):
    # ax.axis('off')
    ax.set_title('channel {}'.format(i + 1))
    _ = ax.imshow(img[i, :, :], cmap='gray')
'''

In [0]:
# img.size()

In [0]:
# print(str_stats(img.numpy().reshape(-1)))

In [0]:
# less analysis/recursion/data/train.csv

In [0]:
# %cd analysis/recursion/data

In [0]:
# !ls -l

In [0]:
# !chmod 700 *

In [0]:
# !unzip train.csv.zip

In [0]:
%cd analysis/recursion/data

/gdrive/My Drive/analysis/recursion/data


In [0]:
df_train = pd.read_csv('train.csv')
df_train.head()

,id_code,experiment,plate,well,sirna
0,HEPG2-01_1_B03,HEPG2-01,1,B03,513
1,HEPG2-01_1_B04,HEPG2-01,1,B04,840
2,HEPG2-01_1_B05,HEPG2-01,1,B05,1020
3,HEPG2-01_1_B06,HEPG2-01,1,B06,254
4,HEPG2-01_1_B07,HEPG2-01,1,B07,144


In [0]:
def load_6_images(dir_name, image_names):
    images = []
    for image_name in image_names:
        image_path = os.path.join('data_raw', 'train2', dir_name, image_name)
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            
        if image is None:
            raise ValueError('Not found image: %s' % image_path)
        images.append(image)
    images = np.stack(images, axis=2)
    return images

In [0]:
def convert_png(rcd):
    experiment = rcd['experiment']
    plate = 'Plate%d' % rcd['plate']
    well = rcd['well']
    dir_name = os.path.join(experiment, plate)    

    sites = [1, 2]
    for site in sites:
        file_names = ['%s_s%d_w%d.png' % (well, site, i+1) for i in range(6)]
        
        dest_file_name = '%s_s%d.pkl' % (well, site)
        dest_dir_name = os.path.join('data_pkl', 'train', dir_name)
        dest_file_path = os.path.join(dest_dir_name, dest_file_name)
        # make dir
        os.makedirs(dest_dir_name, exist_ok=True)

        image = load_6_images(dir_name, file_names)

        # print('write to %s' % dest_file_path)
        joblib.dump(image, dest_file_path)

    return 0

In [0]:
# 12738/36515

In [0]:
_ = [convert_png(df_train.iloc[i]) for i in tqdm(range(12737, len(df_train)))]

  0%|          | 0/23778 [00:00<?, ?it/s]

ValueError: ignored

In [0]:
!ls data_raw/train2/HUVEC-05/Plate2

In [0]:
df_train.head()

,id_code,experiment,plate,well,sirna
0,HEPG2-01_1_B03,HEPG2-01,1,B03,513
1,HEPG2-01_1_B04,HEPG2-01,1,B04,840
2,HEPG2-01_1_B05,HEPG2-01,1,B05,1020
3,HEPG2-01_1_B06,HEPG2-01,1,B06,254
4,HEPG2-01_1_B07,HEPG2-01,1,B07,144
